# Projeto 1 - Backtest do modelo de investimento Magic Formula.


### Desafio:

* Testar se a regra de investimento da fórmula mágica, do Joel Grenblatt, funcionou no Brasil nos últimos anos.


### Passo a passo:

   **Passo 1** - Importar as bibliotecas.
   
   **Passo 2** - Baixar os dados disponibilizados.
   
   **Passo 3** - Pegar as datas que iremos balancear a carteira.
   
   **Passo 4** - Filtrar  a liquidez.
   
   **Passo 5** - Cria o ranking dos indicadores.
   
   **Passo 6** - Cria as carteiras.
   
   **Passo 7** - Calcula a rentabilidade por carteira.
   
   **Passo 8** - Calcula a rentabilidade do modelo.

   **Passo 9** - Calcula a rentabilidade do ibovespa no mesmo período.

   **Passo 10** - Analisar os resultados.
   
-------------------

# Passo 1: Importando os módulos necessários

In [ ]:
!pip install pandas

In [ ]:
!pip install quantstats

In [ ]:
import pandas as pd
import quantstats as qs

# Passo 2: Baixar os dados dispobinilizados.

In [ ]:
dados = pd.read_csv('dados_empresas.csv')

# Passo 3: Filtrar liquidez.

In [ ]:
dados = dados[dados['volume_negociado'] > 1000000]

# Passo 4: Calcula os retornos mensais das empresas.

In [ ]:
dados['retorno'] = dados.groupby('ticker')['preco_fechamento_ajustado'].pct_change()
dados['retorno'] = dados.groupby('ticker')['retorno'].shift(-1)

In [ ]:
dados

# Passo 5: Cria o ranking dos indicadores.

In [ ]:
dados['ranking_ebit_ev'] = dados.groupby('data')['ebit_ev'].rank(ascending = False)
dados['ranking_roic'] = dados.groupby('data')['roic'].rank(ascending = False)

dados['ranking_final'] = dados['ranking_ebit_ev'] + dados['ranking_roic']
dados['ranking_final'] = dados.groupby('data')['ranking_final'].rank()

# Passo 6: Cria as carteiras.

In [ ]:
dados = dados[dados['ranking_final'] <= 10]

# Passo 7: Calcula a rentabilidade por carteira.

In [ ]:
rentabilidade_por_carteiras = dados.groupby('data')['retorno'].mean()
rentabilidade_por_carteiras = rentabilidade_por_carteiras.to_frame()

rentabilidade_por_carteiras

# Passo 8: Calcula a rentabilidade do modelo.

In [ ]:
rentabilidade_por_carteiras['modelo'] = (1 + rentabilidade_por_carteiras['retorno']).cumprod() - 1

rentabilidade_por_carteiras = rentabilidade_por_carteiras.shift(1)
rentabilidade_por_carteiras = rentabilidade_por_carteiras.dropna()

rentabilidade_por_carteiras

# Passo 9: Calcula a rentabilidade do ibovespa no mesmo período.

In [ ]:
ibov = pd.read_csv('ibov.csv')

retornos_ibov = ibov['fechamento'].pct_change().dropna()
retornos_ibov_acum = (1 + retornos_ibov).cumprod() - 1
rentabilidade_por_carteiras['ibovespa'] = retornos_ibov_acum.values

rentabilidade_por_carteiras

In [ ]:
rentabilidade_por_carteiras = rentabilidade_por_carteiras.drop('retorno', axis = 1)

rentabilidade_por_carteiras

# Passo 10: Analisar os resultados.

In [ ]:
qs.extend_pandas()

rentabilidade_por_carteiras.index = pd.to_datetime(rentabilidade_por_carteiras.index)
rentabilidade_por_carteiras

In [ ]:
rentabilidade_por_carteiras['modelo'].plot_monthly_heatmap()
rentabilidade_por_carteiras['ibovespa'].plot_monthly_heatmap()

In [ ]:
rentabilidade_por_carteiras.plot()

In [ ]:
rentabilidade_ao_ano = (1 + rentabilidade_por_carteiras.loc['2023-06-30', 'modelo']) ** (1/10.66) - 1

rentabilidade_ao_ano